# RNN pour l'analyse de sentiments

Nous allons maintenant faire une analyse de sentiments sur le même jeu de données en utilisant les réseaux de neurones récurrents.

## Réseaux de neurones récurrents

Les réseaux de neurones récurrents ou (RNN), sont souvent utilisés pour analyser des séquences.
En effet, dans les réseaux de neurones généraux, un input est traité par un certain nombre de couches et un output est produit à la sortie, avec l'hypothèse que deux inputs successifs sont indépendants.
Cependant, cette hypothèse n'est pas correcte dans un certain nombre de scénarios. 
Par exemple, si on veut prédire le mot suivant dans une séquence, il est indispensable de considérer la dépendance des observations précédentes.
    
Dans notre cas, le modèle RNN prend une séquence de mots $X=\{x_1, ..., x_T\}$, une à la fois, et produit un état caché $h$, pour chaque mot.
On utilise le RNN en lui donnant le mot courant $x_t$ ainsi que l'état caché du mot précédent, $h_{t-1}$, pour produire l'état caché suivant, $h_t$.


$$h_t = \text{RNN}(x_t, h_{t-1})$$


Une fois que l'on a notre état caché final, $h_T$, obtenu après avoir donné le dernier mot de la séquence $x_T$ au modèle, on le donne à une couche linéaire $f$, (qui s'appelle également fully connected layer), pour recevoir notre sentiment prédit, $\hat{y} = f(h_T)$.

<center> <img src="RNN.png" alt="drawing" width="700"/>
        
        
Cette illustration montre un exemple de phrase, avec le RNN prédisant 0, c'est-à-dire que le sentiment est négatif. Le RNN est représenté en orange et la couche linéaire est en gris. On utilise le même RNN pour chaque mot, c'est-à-dire qu'il a les mêmes paramètres. L'état initial caché $h_0$, est un tensor initialisé à zéro.

## Préparation des données

Dans ce notebook, on utilise la librairie torch et TorchText.

TorchText a une méthode `Field` qui sert à définir comment les données brutes doivent être traitées.

La méthode `TEXT` définit comment les commentaires doivent être traités, et `LABEL` comment les labels doivent être traités. 
Ces méthodes comportent plusieurs paramètres qui sont décrits dans la 
<a href="https://torchtext.readthedocs.io/en/latest/data.html#field">documentation</a>.

In [1]:
import pandas as pd
import torch
from torchtext import data

# pour la reproductibilité
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(sequential=True,lower=True, tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

On sépare les données en données de train, de test et de validation.

In [2]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')

Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


On affiche un exemple d'un commentaire "tokénisé" avec le label correspondant.

In [3]:
print(vars(train_data.examples[1]))

{'text': ['i', 'really', 'wonder', 'how', 'this', 'show', 'plays', 'in', 'the', 'u.k.', 'and', 'the', 'rest', 'of', 'europe', '.', 'it', 'is', 'so', 'self', '-', 'loathing', 'about', 'being', 'an', 'american(particularly', 'white', 'americans', ')', '.', 'that', 'could', 'be', 'a', 'big', 'reason', 'for', 'some', 'of', 'the', 'venom', 'and', 'vitriol', 'expressed', 'here', 'on', 'this', 'board', '.', 'i', 'love', 'the', 'show', 'but', 'it', 'is', 'with', 'some', 'reservations', 'and', 'i', 'feel', 'it', 'took', 'the', 'easy', 'way', 'out', 'by', 'spoiler', 'spoiler:<br', '/><br', '/>crashing', 'and', 'burning', 'everything', 'at', 'the', 'end', 'of', 'the', 'second', 'season', '.', 'julie', 'slammed', 'the', 'door', 'shut', 'on', 'any', 'hope', 'of', 'reviving', 'the', 'show', 'unless', 'her', 'character', 'lands', 'on', 'a', 'haystack', 'in', 'the', 'middle', 'of', 'some', 'farm', '.', 'who', 'knows', '?', 'it', 'was', 'a', 'funny', ',', 'raunchy', 'and', 'on', 'occasion', ',', 'kinda

In [4]:
TEXT.build_vocab(train_data)
print(f"Nombre de tokens unique dans le TEXT: {len(TEXT.vocab)}") 

Nombre de tokens unique dans le TEXT: 89987


Le vocabulaire est composé de 89987 mots. On le réduit en ne prennant que 25000 mots pour ne pas que la dimension de la matrice d'embedding soit trop grande.

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Nombre de tokens unique dans le TEXT: {len(TEXT.vocab)}") 
print(f"Nombre unique de LABEL: {len(LABEL.vocab)}")

Nombre de tokens unique dans le TEXT: 25002
Nombre unique de LABEL: 2


Les deux valeurs supplémentaires dans le vocabulaire du TEXT sont les tokens `<unk>` et `<pad>`. Si un mot de l'échantillon de validation ou de test n'est pas dans le vocabulaire, alors celui-ci est remplacé par `<unk>`. Le token `<pad>` est utilisé pour que les séquences d'un même batch soient de mêmes longueurs. 

On affiche ci-dessous les 20 mots les plus fréquents.

In [6]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 274223), (',', 239861), ('.', 204258), ('and', 135511), ('a', 135466), ('of', 121892), ('to', 112826), ('is', 92798), ('in', 77926), ('it', 74198), ('i', 60943), ('that', 60269), ('this', 55592), ('"', 54672), ("'s", 54540), ('-', 45599), ('\n', 42455), ('as', 39521), ('with', 37130), ('was', 36791)]


Les valeurs du vocabulaire de LABEL sont 0 pour positif et 1 pour négatif.

In [7]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


La dernière étape de la préparation des données consiste à créer les itérateurs. Nous les parcourons dans la boucle d'apprentissage / d'évaluation, et ils retournent un batch à chaque itération.

On utilise `BucketIterator` qui est un itérateur qui renverra un batch de séquences de mêmes longueurs grâce au padding.

In [8]:
# utilisation du GPU si possible 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort = False)

## Construction du modèle

Le modèle RNN utilisé se compose des couches suivantes : 

- _embedding_ : utilisé pour transformer notre vecteur one hot encoder (dont la plupart des éléments sont des 0) en un vecteur embedding dense (dense car la dimensionnalité est beaucoup plus petite et tous les éléments sont des nombres réels). De plus, les mots qui ont un impact similaire sur le sentiment de la revue sont mappés de manière rapprochée dans cet espace vectoriel dense.
- _RNN_ : prend le vecteur dense et le précédent état caché $h_{t-1}$, et calcule l'état caché suivant $h_t$.
- _linéaire_ : prend le dernier état caché, le met dans un couche fully connected $f(h_T)$ qui le transfome en output prédit.


Par ailleurs, le modèle considéré est bidirectionnel, il peut être représenté comme ceci.


<center> <img src="birnn.png" alt="drawing" width="600"/>


En plus d'avoir un RNN traitant les mots de la phrase du premier au dernier (un RNN forward), nous avons un deuxième RNN traitant les mots de la phrase du dernier au premier (un RNN backward). Au pas de temps $t$, le RNN forward traite le mot $ x_t $, et le RNN backward traite le mot $ x_ {T-t + 1}

Chaque batch, `text` est un tensor de dimension _**[taille séquence, taille batch]**_. C'est un batch de commentaires, dont les mots sont convertis en représentation one-hot encoder. PyTorch stocke  un vecteur one-hot comme sa valeur d'index.

Le batch de l'input est ensuite passé à travers la couche embedding pour obtenir une représentation de vecteur dense de nos commentaires.
`embedded` est un tensor de taille _**[taille séquence, taille batch, dim embedding]**_.

`embedded`est ensuite introduit dans la couche RNN. Dans certains frameworks, vous devez alimenter l'état caché initial, $h_0$, dans le RNN, cependant dans PyTorch, si aucun état caché initial n'est passé comme argument, il prend par défaut un tenseur de valeurs zéros.

The RNN retourne 2 tenseurs, `output` de taille _**[taille séquence, taille batch, dim état caché*2]**_ et `hidden` de taille _**[num direction, taille batch, dim état caché]**_.

`output` est la concaténation de l'état caché de chaque pas de temps. Comme nous voulons les états cachés avant et arrière de la couche finale (supérieure), nous obtenons les deux couches cachées supérieures de la première dimension, hidden [-2,:,:] et hidden [-1,:,:], et les concaténons ensemble ( _**[taille batch, dim état caché*2]**_ ) avant de les transmettre à la couche linéaire, pour produire une prédiction.

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,bidirectional):
  
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim) 
        self.rnn = nn.RNN(embedding_dim, hidden_dim, bidirectional=bidirectional)   
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text) 
        output, hidden = self.rnn(embedded) 
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        
        return self.fc(hidden)


On crée alors notre modèle.

 - La dimension de l'input est la taille du vocabulaire. 
 - La dimension de l'embedding est la taille des vecteurs denses. Elle est généralement de 50 à 250, mais elle dépend de la taille du vocabulaire.
 - La dimension de hidden est la dimension des états cachés. Elle vaut généralement 100 à 500, mais dépend de la taille du vocabulaire, des vecteurs denses et la compléxité de la tâche. 
 - La dimension de l'output est le nombre de classes, ici, la valeur de l'output est entre 0 et 1 et donc est de dimension 1 car l'output est un scalaire.

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
BIDIRECTIONAL = True
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,BIDIRECTIONAL )

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner')

Le modèle a 2,684,009 paramètres à entraîner


## Entraîner le modèle

On utilise l'optimiseur Adam qui ne prend en compte seulement comme paramètres les paramètres du modèle qui seront actualisés par l'optimiseur.

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

On définit ensuite la fonction de perte qui est ici la _binary cross entropy with logits_. 

In [13]:
criterion = nn.BCEWithLogitsLoss()

En utilisant `.to`, nous pouvons placer le modèle et le critère sur le GPU (si nous en avons un).

In [14]:
model = model.to(device)
criterion = criterion.to(device)

Notre fonction criterion calcule la perte, cependant nous devons écrire notre fonction pour calculer l'accuracy, le recall et le f1score.

 - Accuracy : 
$$accuracy = \frac{TP+TN}{TP+TN+FP+FN}\,,$$

 - Recall :
$$recall = \frac{TP}{TP+FN}\,,$$

 - F1 score : 
$$f1score = 2*\frac{precision*recall}{precision+recall}\,,$$
 
avec $TP$ : vrais positifs, $TN$ : vrais négatifs, $FP$ : faux positifs, $FN$ : faux négatifs et $precision = \frac{TP}{TP+FP}$

In [27]:
def binary_accuracy(preds, y):
    """
    Retourne l'accuracy par batch
    """
    #arrondi la prédiction à l'entier le plus proche
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def recall(preds, y):
    '''
    Retourne le recall
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float()       
    
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    if (tp + fn) == 0:
        recall = torch.zeros(1)
        
    recall = tp / (tp + fn)
    return recall



def f1_loss(preds, y):
    '''
    Retourne le score F1
    '''  
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    if (tp + fn) == 0 or (tp + fp) == 0 or (recall + precision == 0):
        f1 = torch.zeros(1)
    else:
        f1 = 2* (precision*recall) / (precision + recall)
    
    return f1

La fonction `train` itère sur tous les exemples, un batch à la fois.

`model.train()`est utilisé pour mettre le modèle en "mode entraînement", qui active _dropout_ et _batch normalization_, qui ne sont pas utilisés ici.

Pour chaque batch, on met à zéro le gradient. Chaque paramètre dans un modèle a un attribut `grad` qui stocke le gradient calculé par `criterion`. PyTorch ne met pas à zéro les gradients automatiquement.

Nous introduisons ensuite le batch de phrases, `batch.text`, dans le modèle. Le `squeeze` est nécessaire car les prédictions sont initialement de taille _**[taille batch, 1]**_, et nous devons supprimer la dimension de taille 1 car PyTorch s'attend à ce que les prédictions entrées dans notre fonction de critère soient de taille _**[taille batch]**_.
La perte et la précision sont ensuite calculées à l'aide de nos prédictions et des labels, `batch.label`, la perte étant moyennée sur toutes les séquences du batch.

Nous calculons le gradient de chaque paramètre avec `loss.backward ()`, puis mettons à jour les paramètres en utilisant les gradients et l'algorithme d'optimisation avec `optimizer.step ()`.

Enfin, nous retournons la perte, la précision, le recall et le F1 score moyennés sur toute l'époque. 


In [16]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
               
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        rec = recall(predictions, batch.label)
        f1 = f1_loss(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_rec += rec.item()
        epoch_f1 += f1.item()        
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)

La fonction `evaluate` est similaire à` train`, avec quelques modifications car vous ne voulez pas mettre à jour les paramètres lors de l'évaluation.

`model.eval ()` met le modèle en "mode d'évaluation", ceci désactive _dropout_ et _batch normalization_.

Aucun gradient n'est calculé sur les opérations PyTorch à l'intérieur du bloc `with no_grad ()`. Cela réduit l'utilisation de la mémoire et accélère le calcul.

Le reste de la fonction est identique à `train`, avec la suppression de` optimizer.zero_grad () `,` loss.backward () `et` optimizer.step () `, car nous ne mettons pas à jour les paramètres du modèle lors de l'évaluation.

In [17]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model.forward(batch.text).squeeze(1) 
            loss = criterion(predictions, batch.label)    

            acc = binary_accuracy(predictions, batch.label)
            rec = recall(predictions, batch.label)
            f1 = f1_loss(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_rec += rec.item()
            epoch_f1 += f1.item() 
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)

Nous entraînons ensuite le modèle sur plusieurs époques, une époque étant un passage complet à travers toutes les séquences dans les ensembles d'apprentissage et de validation.

À chaque époque, si la perte de validation est la meilleure que nous ayons vue jusqu'à présent, nous enregistrerons les paramètres du modèle, puis une fois l'entraînement terminé, nous utiliserons ce modèle sur l'ensemble de test.

In [28]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_rec, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_rec, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Recall: {train_rec*100:.2f}% | Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. Recall: {valid_rec*100:.2f}%  | Val. F1: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 7s
	Train Loss: 0.662 | Train Acc: 59.96% | Train Recall: 34.19% | Train F1: 40.36%
	 Val. Loss: 0.680 |  Val. Acc: 57.82% | Val. Recall: 37.87%  | Val. F1: 44.33%
Epoch: 02 | Epoch Time: 1m 2s
	Train Loss: 0.640 | Train Acc: 63.44% | Train Recall: 38.76% | Train F1: 45.87%
	 Val. Loss: 0.659 |  Val. Acc: 61.11% | Val. Recall: 43.71%  | Val. F1: 49.65%
Epoch: 03 | Epoch Time: 1m 10s
	Train Loss: 0.605 | Train Acc: 67.98% | Train Recall: 41.23% | Train F1: 49.87%
	 Val. Loss: 0.645 |  Val. Acc: 63.83% | Val. Recall: 43.32%  | Val. F1: 50.61%
Epoch: 04 | Epoch Time: 1m 13s
	Train Loss: 0.588 | Train Acc: 69.57% | Train Recall: 39.65% | Train F1: 49.28%
	 Val. Loss: 0.744 |  Val. Acc: 56.00% | Val. Recall: 43.65%  | Val. F1: 40.93%
Epoch: 05 | Epoch Time: 1m 7s
	Train Loss: 0.576 | Train Acc: 70.07% | Train Recall: 40.28% | Train F1: 49.52%
	 Val. Loss: 0.648 |  Val. Acc: 64.30% | Val. Recall: 45.37%  | Val. F1: 51.97%
Epoch: 06 | Epoch Time: 1m 17s
	Train Loss:

## Tester le modèle

In [29]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc, test_rec, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%| Test Recall: {test_rec*100:.2f}%  | Test F1: {test_f1*100:.2f}%')

Test Loss: 0.648 | Test Acc: 68.21%| Test Recall: 40.61%  | Test F1: 50.41%


### Inférence

Notre fonction predict_sentiment fait plusieurs choses:

 - définit le modèle en mode d'évaluation
 - tokenise la phrase, c'est-à-dire la divise d'une chaîne brute en une liste de jetons
 - indexe les jetons en les convertissant en leur représentation entière à partir de notre vocabulaire
 - obtient la longueur de notre séquence
 - convertit les index, qui sont une liste Python en un tenseur PyTorch
 - ajouter une dimension de lot en relâchant
 - convertit la longueur en un tenseur
 - écrase la prédiction de sortie d'un nombre réel compris entre 0 et 1 avec la fonction sigmoïde
 - convertit le tenseur contenant une valeur unique en un entier avec la méthode item ()

Nous nous attendons à ce que les avis avec un sentiment négatif renvoient une valeur proche de 0 et les avis positifs renvoient une valeur proche de 1.

In [30]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [31]:
predict_sentiment(model, "This film is terrible")

0.9002652764320374

In [32]:
predict_sentiment(model, "This film is great")

0.07549090683460236

On remarque que ce modèle nous donne de faible résultats. En utilisant un modèle plus avancé on obtiendrait de meilleurs résultats.

## Réference : 

 - https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
